In [10]:
# install dependencies
!pip install --upgrade opencv-python
# !pip install imutils
# !python -m pip install --upgrade pip
# !pip install numpy
# !pip install argparse
# !pip install os_sys
# !pip install os-win

In [1]:
# import libraries 
import cv2 as cv
import argparse
import sys
import numpy as np
import os
from imutils import paths
import time
import ntpath

In [2]:
# Initialize the parameters
confThreshold = 0.1  #Confidence threshold
nmsThreshold = 0.4  #Non-maximum suppression threshold

inpWidth = 416  #Width of network's input image
inpHeight = 416 #Height of network's input image

predicted_list =[]
time_taken =[]

classesFile = "classes.names"; # Load names of classes

classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

# Give the configuration and weight files for the model and load the network using them.

modelConfiguration = "darknet-yolov3.cfg";
modelWeights = "weights/darknet-yolov3_final.weights";

net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

In [3]:
classes # there is only one class in class.names

['monkey']

In [4]:
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [5]:
# Draw the predicted bounding box
def drawPred(classId,conf, left, top, right, bottom,frame):
    h=frame.shape[0]
    w=frame.shape[1]
    left=int(left*w)
    top=int(top*h)
    right=int(right*w)
    bottom=int(bottom*h)

    # Draw a bounding box.
    cv.rectangle(frame,(left, top),(right, bottom), (0, 255, 0), 3)
    label = '%.2f' % conf
        
    # Get the label for the class name and its confidence
    if classes:
        assert(classId < len(classes))
        label = '%s:%s' % (classes[classId], label)
    #Display the label at the top of the bounding box
    labelSize, baseLine = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine), (0, 0, 255), cv.FILLED)
    cv.putText(frame, label, (left, top), cv.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 2)

In [6]:
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    classIds = []
    confidences = []
    boxes = []
    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    annotation_list=[]
    for out in outs:
        for detection in out:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    indices = cv.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    flag=1
    for i in indices:
        print("Monkey Detected")
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        annotation_list.append([classIds[i],confidences[i], left/frameWidth, top/frameHeight, (left + width)/frameWidth, (top + height)/frameHeight])
    return annotation_list

In [7]:
# import re
# def sorted_aphanumeric(data):
#     convert = lambda text: int(text) if text.isdigit() else text.lower()
#     alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
#     return sorted(data, key=alphanum_key)

In [8]:

# Process inputs
winName = 'Deep learning object detection in OpenCV'
cv.namedWindow(winName, cv.WINDOW_NORMAL)

inputPath ="InputImages/"
outputPath="OutputImages/"
imagelistInput = os.listdir(inputPath)

# imagePaths = sorted_aphanumeric(imagePaths)

for file_name in imagelistInput:
    cap = cv.VideoCapture(inputPath+file_name)
    tok =time.time()
    box_filename=file_name.split(".")[0]+".txt"
    print(file_name)
    while cv.waitKey(1) < 0:
    
        # get frame from the video
        hasFrame, frame = cap.read()

        # Stop the program if reached end of video
        if not hasFrame:
            print("Done processing !!!\n")
            break

        # Create a 4D blob from a frame.
        blob = cv.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)

        # Sets the input to the network
        net.setInput(blob)

        # Runs the forward pass to get output of the output layers
        outs = net.forward(getOutputsNames(net))

        # Remove the bounding boxes with low confidence
        annotation_list=postprocess(frame, outs)
        f=open(outputPath+"out_"+box_filename,"w")
        for j in annotation_list:
            drawPred(0,j[1],j[2],j[3],j[4],j[5],frame)
            annotation=str(0) +" "
            for k in j[2:]:
                annotation=annotation+str(k)+" "
            f.write(annotation)    
            f.write("\n")
            annotation=""
        f.close()
        outputFile=outputPath+"out_"+ file_name
        cv.imwrite(outputFile,frame.astype(np.uint8))

        # Put efficiency information. The function getPerfProfile returns the overall time for inference(t) and the timings for each of the layers(in layersTimes)
        t, _ = net.getPerfProfile()
        label = 'Inference time: %.2f ms' % (t * 1000.0 / cv.getTickFrequency())
        
        tik=time.time()
        print("Time : ", '%.3f'%(tik-tok))
        cv.imshow(winName, frame)
       

cv.destroyAllWindows()
cv.VideoCapture(0).release()

16b5881aa0469a10.jpg
Monkey Detected
Monkey Detected
Monkey Detected
Monkey Detected
Time :  3.125
Done processing !!!

monkey1.jpg
Monkey Detected
Time :  0.937
Done processing !!!

monkey_s.jpg
Monkey Detected
Time :  1.027
Done processing !!!

test.jpeg
Time :  1.020
Done processing !!!

yolo_out_py17.jpg
Monkey Detected
Time :  1.071
Done processing !!!



In [9]:
# For live detection
winName = 'Deep learning object detection in OpenCV'
cv.namedWindow(winName, cv.WINDOW_NORMAL)

cap = cv.VideoCapture(0)
cap.set(cv.CAP_PROP_FRAME_WIDTH, 416)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 416)
while cv.waitKey(1) < 0:
    
    # get frame from the video
    hasFrame, frame = cap.read()

    # Stop the program if reached end of video
    if not hasFrame:
        print("Done processing !!!\n")
        break

    # Create a 4D blob from a frame.
    blob = cv.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)

    # Sets the input to the network
    net.setInput(blob)

    # Runs the forward pass to get output of the output layers
    outs = net.forward(getOutputsNames(net))
    
    # Remove the bounding boxes with low confidence
    annotation_list=postprocess(frame, outs)    
    for j in annotation_list:
        drawPred(0,j[1],j[2],j[3],j[4],j[5],frame)
        
    cv.imshow(winName, frame)
       
cv.destroyAllWindows()
cv.VideoCapture(0).release()

Monkey Detected
Monkey Detected
Monkey Detected
Monkey Detected
Monkey Detected
Monkey Detected
Monkey Detected
Monkey Detected


KeyboardInterrupt: 